In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd

from bs4 import BeautifulSoup
import requests
import time, os

pd.options.mode.chained_assignment = None

### Import 2021 demographic data (DATA-SMART CITY SAPPORO)

In [ ]:
# Import CSV from Sapporo municipal gov't, after editing in Excel (ensure 'header=1')
# Source: DATA-SMART CITY SAPPORO (町名・条丁目別世帯数及び男女別人口 令和3年（2021年）4月1日現在.csv)

df = pd.read_csv('町名・条丁目別_export.csv', header=1)
df.columns = ['Ward', 'Block', 'N_Households_2021',
              "Population_2021", "Male", "Female"]

In [ ]:
# For checking individual entries
# df[df['町条丁目'] == "宮の森一条１０丁目"] 

### Import shape data (Geoshape Repository)

In [ ]:
# Import SHP from Geoshape Repository > 国勢調査町丁・字等別境界データ
# Source: https://geoshape.ex.nii.ac.jp/ka/

dfgeo_japan = gpd.read_file('h27ka01.shp')
dfgeo_sapp = dfgeo_japan.iloc[:5796, :]  #  清田区→里塚緑ケ丘１２丁目 = last entry for Sapporo City

In [ ]:
# For checking individual entries
# dfgeo_sapp[dfgeo_sapp['S_NAME'] == "宮の森（番地）"]

In [ ]:
# Save Sapporo data as CSV

dfgeo_sapp.to_csv('GISrefdata_for_checking.csv')

In [ ]:
# Reduce feature for data analysis

dfgeo_sapp_cut = dfgeo_sapp[["KEY_CODE",   # unique ID for join in Tableau
                            "PREF_NAME", "CITY_NAME", "S_NAME", "KIGO_E",
                            "AREA", "JINKO", "SETAI",
                            "X_CODE", "Y_CODE"]]

dfgeo_sapp_cut.rename(columns={'JINKO': 'Population_2015', 'SETAI': 'N_Households_2015', 'KIGO_E': 'Kigo_E',
                              'PREF_NAME': 'Pref_ref', 'CITY_NAME': 'Ward_ref', 'S_NAME': 'Block_ref',
                              'AREA': 'Area', "X_CODE": 'Lat', 'Y_CODE': 'Lng'}, inplace=True)

### Join datasets on neighborhood

In [ ]:
# Join datasets on ['Ward', 'Block'], ['CITY_NAME', "S_NAME"]

joined_df = pd.merge(df, dfgeo_sapp_cut, left_on=['Ward', 'Block'], right_on=['Ward_ref', 'Block_ref'], how='left')
# new_df = joined_df[joined_df['KEY_CODE'].notna()]
new_df = joined_df[["KEY_CODE",   # unique ID for join in Tableau
                    "Ward", "Block", "Pref_ref", "Ward_ref", "Block_ref", "Kigo_E",
                    "N_Households_2015", "N_Households_2021",
                    "Population_2021", "Population_2015",
                    "Male", "Female", "Area", "Lat", "Lng"]]

### Derived features

In [ ]:
# Population density
new_df['PopDen_2015'] = new_df['Population_2015'] / new_df['Area']
new_df['PopDen_2021'] = new_df['Population_2021'] / new_df['Area']

# Gender ratio
new_df['Sex_ratio'] = new_df['Female'] / (new_df['Female'] + new_df['Male'])

# People per household
# Greater values = Families, multigenerational households
new_df["Household_size_2015"] = new_df['Population_2015'] / new_df['N_Households_2015']
new_df["Household_size_2021"] = new_df['Population_2021'] / new_df['N_Households_2021']

new_df

In [ ]:
# Export for checking in Numbers
# (10-Oct) Some gaps, but mostly clean

new_df.to_csv('joined_dataframe_for_checking_in_numbers.csv')

In [ ]:
# Remove rows that failed to match (<1% of rows; fix later)

new_df = new_df[new_df['KEY_CODE'].notna()].reset_index(drop=True)

# Convert ID to string for matching in Tableau

new_df['KEY_CODE'] = [str(id_no) for id_no in new_df.KEY_CODE]
new_df

In [ ]:
new_df.to_csv('sapporo_df_plus_geodata.csv')

### EDA

In [ ]:
# Mask data here
# Limit only districts with population >= 10
df_eda = new_df[new_df['Population_2021'] >= 10]

In [ ]:
# Top five blocks with greatest gender imbalance in residents

df_eda.sort_values(by='Sex_ratio', ascending=True)

In [ ]:
# Top five blocks with largest/smallest households

df_eda.sort_values(by='Household_size_2021', ascending=True)